#### **Projekt1**

Pre každé z troch trénovaní vyčíslite úspešnosť klasifikácie na testovacích dátach. Spolu so zobrazením kontingenčnej matice pre behy, matica najlepšieho behu sa vyberie

In [ ]:
import pickle
import numpy as np
import torch
from pretty_confusion_matrix import pp_matrix_from_data
from models_architecture import *
from dataset_architecture import *
from matplotlib import pyplot as plt
from torchvision import utils
from torchvision import transforms
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

*Launch notebook for the respective network*

#### Resnet18

In [ ]:
model = resnet18_architecture()
name = model.__class__.__name__
BATCH_SIZE = 32

#### Alexnet

In [ ]:
model = alexnet_architecture()
name = model.__class__.__name__
BATCH_SIZE = 128

#### Inception

In [ ]:
model = inception_architecture()
name = model.__class__.__name__
BATCH_SIZE = 128

#### Mobilenet

In [ ]:
model = inception_architecture()
name = model.__class__.__name__
BATCH_SIZE = 128

### Routines

*this needs to be launched befor going further*

In [ ]:
def test_model(model, testloader):
    model.eval()  # activate evaulation mode, some layers behave differently
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        model.cuda()
    total = 0
    correct = 0
    final_predicted = []
    final_labels = []
    for inputs, labels in iter(testloader):
        if use_cuda:
            inputs = inputs.cuda()
            labels = labels.cuda()
        with torch.no_grad():
            outputs_batch = model(inputs)

        _, predicted = torch.max(outputs_batch.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        final_predicted += predicted.tolist()
        final_labels += labels.tolist()
    print(
        "Accuracy of the network on the test images: %0.2f %%" % (100 * correct / total)
    )

    return final_predicted, final_labels

In [ ]:
def plot_matrix(net_predicted,net_labels):
    labels = [i for i in range(25)]
    array = confusion_matrix(net_labels, net_predicted, labels=labels)
    df_cm = pd.DataFrame(array, range(25), range(25))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 10}) 
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.title("Confusion Matrix")
    plt.show()
    
  #  pp_matrix_from_data(net_labels, net_predicted, columns=labels, cmap="gnuplot2")

In [ ]:
path = 'trained_models'
TEST_SPLIT=0.2

transform = transforms.Compose(
        [
            transforms.Resize(256, transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.ConvertImageDtype(dtype=torch.float32),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ]
    )

_, testloader = get_architectural_dataset(
        #root_path="architectural-styles-dataset/",
        root_path="C:\\Repositories\\hns\\Dataset_Architektura\\architectural-styles-dataset", 
        transform=transform,
        batch_sz=BATCH_SIZE,
        test=TEST_SPLIT,
    )

#runs = ["1", "2", "3"]
runs = ["1"]

# Iterate over each string in the list
for run in runs:
    print(run)
    model.load_state_dict(torch.load(f"{path}/{name}/{run}/model.pt"))
    predicted,labels=test_model(model,testloader)
    
    %matplotlib qt
    plot_matrix(predicted,labels)
    


